# data preprocessing

In [ ]:
import numpy as np
import pandas as pd
import torch
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
# deleting the 'id' column  and Unnamed: 32 column
df.drop(columns=['id', 'Unnamed: 32'], inplace =True)

from sklearn.model_selection import train_test_split
x_train, x_test ,y_train, y_test = train_test_split(x,y, test_size=0.2)

from sklearn.preprocessing import StandardScaler
scaler =StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

import torch
x_train_tensor  = torch.from_numpy(x_train)
x_test_tensor   = torch.from_numpy(x_test)
y_train_tensor  = torch.from_numpy(y_train)
y_test_tensor   = torch.from_numpy(y_test)  

# Model train